## 0.0 - IMPORTAÇÕES DE PACOTES E MÓDULOS

In [1]:
import pandas as pd
import numpy as np
import inflection
import seaborn as sb
from matplotlib import pyplot as plt
import inflection
import warnings

In [2]:
warnings.filterwarnings('ignore')

## 1.0 - CARREGAMENTO DO DATASET

In [3]:
df_train_raw = pd.read_csv('data/train.csv', low_memory=False)
df_store_raw = pd.read_csv('data/store.csv', low_memory=False)

In [4]:
df_train_raw.head(1)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1


In [5]:
df_store_raw.head(1)

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


### 1.1 - Juntando os dois Datasets

In [6]:
df_raw = pd.merge(df_train_raw, df_store_raw, how='left', on='Store')

In [7]:
df_raw.head(1)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


### 1.2 - Transformando o os labels das colunas em Snake Case

In [8]:
columns_old = list(df_raw.columns)

In [9]:
columns_new = map(lambda x : inflection.underscore(x), columns_old)

In [10]:
df_raw.columns = columns_new

In [11]:
df_raw.head(1)

,store,day_of_week,date,sales,customers,open,promo,state_holiday,school_holiday,store_type,assortment,competition_distance,competition_open_since_month,competition_open_since_year,promo2,promo2_since_week,promo2_since_year,promo_interval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


## 2.0 - DESCRIÇÃO DOS DADOS

### 2.0.1 - Copia dos dados de um tópico para outro

In [12]:
df2 = df_raw.copy()

### 2.0.2 - Verificando as dimensões do Dataset

In [13]:
print(f'Número de linhas: {df2.shape[0]}')
print(f'Número de colunas: {df2.shape[1]}')

Número de linhas: 1017209
Número de colunas: 18


### 2.0.3 - Verificando o tipo das variáveis e valores missing

In [14]:
df2.dtypes

### OBS.: Pode ser usado também o comando df2.info() ###

store                             int64
day_of_week                       int64
date                             object
sales                             int64
customers                         int64
open                              int64
promo                             int64
state_holiday                    object
school_holiday                    int64
store_type                       object
assortment                       object
competition_distance            float64
competition_open_since_month    float64
competition_open_since_year     float64
promo2                            int64
promo2_since_week               float64
promo2_since_year               float64
promo_interval                   object
dtype: object

In [15]:
df2.isna().sum() 

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64

#### 2.0.3.1 - Alterando o tipo da variável 'date'

In [16]:
df2.date = pd.to_datetime(df2.date)

In [17]:
print(f'Tipo da variável "date" alterado para: {df2.date.dtypes}') 

Tipo da variável "date" alterado para: datetime64[ns]


#### 2.0.3.2 - Substituindo valores NA

In [18]:
# competition_distance SUBSTITUINDO OS VALORES NA POR 0 PARA INDICAR QUE NÃO EXISTEM CONCORRENTES
df2.competition_distance = df2.competition_distance.fillna(0)

In [19]:
# competition_open_since_month SUBSTITUINDO PELO MÊS DA DATA DE ABERTURA DA LOJA
df2.competition_open_since_month = df2.competition_open_since_month.fillna(df2.date.dt.month)

In [20]:
# competition_open_since_year SUBSTITUINDO PELO ANO DA DATA DE ABERTURA DA LOJA  
df2.competition_open_since_year = df2.competition_open_since_year.fillna(df2.date.dt.year)

In [21]:
# promo2_since_week SUBSTITUINDO PELA DIA DA SEMANA DA DATA DE ABERTURA DA LOJA 
df2.promo2_since_week = df2.promo2_since_week.fillna(df2.date.dt.week)

In [22]:
# promo2_since_year SUBSTITUINDO PELO ANO DA DATA DE ABERTURA DA LOJA 
df2.promo2_since_year = df2.promo2_since_year.fillna(df2.date.dt.year)

In [23]:
# promo_interval  
df2.promo_interval = df2.promo_interval.fillna(0)

# Criando uma coluna chamada month_map e preenchendo com os meses do ano
month_map = {1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'Jun',7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec', }

df2['month_map'] = df2.date.dt.month.map(month_map)

# Criando uma coluna chamada is_promo 
df2['is_promo'] = df2[['promo_interval', 'month_map']].apply( lambda x: 0 if x['promo_interval'] == 0 else 1 if x['month_map'] in x['promo_interval'].split( ',' ) else 0, axis=1 )

In [24]:
df2.head()

,store,day_of_week,date,sales,customers,open,promo,state_holiday,school_holiday,store_type,assortment,competition_distance,competition_open_since_month,competition_open_since_year,promo2,promo2_since_week,promo2_since_year,promo_interval,month_map,is_promo
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,31.0,2015.0,0,Jul,0
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",Jul,1
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",Jul,1
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,31.0,2015.0,0,Jul,0
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,31.0,2015.0,0,Jul,0


#### 2.0.3.3 - Alterando os tipos das variáveis

In [25]:
# competition_open_since_month 
df2.competition_open_since_month = df2.competition_open_since_month.astype('int64')

# competition_open_since_year
df2.competition_open_since_year = df2.competition_open_since_year.astype('int64')

# promo2_since_week
df2.promo2_since_week = df2.promo2_since_week.astype('int64')

# promo2_since_year
df2.promo2_since_year = df2.promo2_since_year.astype('int64')

### 2.0.4 - Estatística Descritiva

#### 2.0.4.1 - Separando dados Numéricos e Categóricos

In [26]:
num_attributes = df2.select_dtypes( include=['int64', 'float64'] )
cat_attributes = df2.select_dtypes( exclude=['int64', 'float64', 'datetime64[ns]'] )

#### 2.0.4.2 - Descrição dos Dados Numéricos

In [27]:
# Central Tendency - mean, meadina 
ct1 = pd.DataFrame( num_attributes.apply( np.mean ) ).T
ct2 = pd.DataFrame( num_attributes.apply( np.median ) ).T

# dispersion - std, min, max, range, skew, kurtosis
d1 = pd.DataFrame( num_attributes.apply( np.std ) ).T 
d2 = pd.DataFrame( num_attributes.apply( min ) ).T 
d3 = pd.DataFrame( num_attributes.apply( max ) ).T 
d4 = pd.DataFrame( num_attributes.apply( lambda x: x.max() - x.min() ) ).T 
d5 = pd.DataFrame( num_attributes.apply( lambda x: x.skew() ) ).T 
d6 = pd.DataFrame( num_attributes.apply( lambda x: x.kurtosis() ) ).T 

# concatenar
m = pd.concat( [d2, d3, d4, ct1, ct2, d1, d5, d6] ).T.reset_index()
m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
m

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,store,1.0,1115.0,1114.0,558.429727,558.0,321.908493,-0.000955,-1.200524
1,day_of_week,1.0,7.0,6.0,3.998341,4.0,1.997390,0.001593,-1.246873
2,sales,0.0,41551.0,41551.0,5773.818972,5744.0,3849.924283,0.641460,1.778375
3,customers,0.0,7388.0,7388.0,633.145946,609.0,464.411506,1.598650,7.091773
4,open,0.0,1.0,1.0,0.830107,1.0,0.375539,-1.758045,1.090723
5,promo,0.0,1.0,1.0,0.381515,0.0,0.485758,0.487838,-1.762018
6,school_holiday,0.0,1.0,1.0,0.178647,0.0,0.383056,1.677842,0.815154
7,competition_distance,0.0,75860.0,75860.0,5415.982074,2320.0,7710.248918,2.931275,13.022556
8,competition_open_since_month,1.0,12.0,11.0,6.786849,7.0,3.311085,-0.042076,-1.232607
9,competition_open_since_year,1900.0,2015.0,115.0,2010.324840,2012.0,5.515591,-7.235657,124.071304


#### 2.0.4.3 - Descrição dos Dados Categóricos

In [28]:
# 'state_holiday'

In [29]:
cat_attributes.columns

Index(['state_holiday', 'store_type', 'assortment', 'promo_interval',
       'month_map'],
      dtype='object')

In [30]:
state_holiday_frequency = cat_attributes.state_holiday.value_counts()

In [31]:
table_frequency = pd.DataFrame(columns=['Frequencia', 'Frequencia_Relativa'], index=['Nenhum', 'Outros', 'Pascoa', 'Natal', 'TOTAL'])

In [32]:
table_frequency.Frequencia[0] = state_holiday_frequency[0]
table_frequency.Frequencia[1] = state_holiday_frequency[1]
table_frequency.Frequencia[2] = state_holiday_frequency[2]
table_frequency.Frequencia[3] = state_holiday_frequency[3]
table_frequency.Frequencia[4] = state_holiday_frequency.sum()

table_frequency.Frequencia_Relativa[0] = round(state_holiday_frequency[0] / table_frequency.Frequencia[4] * 100,2)
table_frequency.Frequencia_Relativa[1] = round(state_holiday_frequency[1] / table_frequency.Frequencia[4] * 100,2)
table_frequency.Frequencia_Relativa[2] = round(state_holiday_frequency[2] / table_frequency.Frequencia[4] * 100,2)
table_frequency.Frequencia_Relativa[3] = round(state_holiday_frequency[3] / table_frequency.Frequencia[4] * 100,2)
table_frequency.Frequencia_Relativa[4] = table_frequency.Frequencia_Relativa.sum()


In [33]:
table_frequency

,Frequencia,Frequencia_Relativa
Nenhum,986159,96.95
Outros,20260,1.99
Pascoa,6690,0.66
Natal,4100,0.4
TOTAL,1017209,100


In [34]:
# 'store_type'

In [35]:
store_type_frequency = cat_attributes.store_type.value_counts()

In [36]:
table_frequency = pd.DataFrame(columns=['Frequencia', 'Frequencia_Relativa'], index=['A', 'B', 'C', 'D', 'TOTAL'])

In [37]:
table_frequency.Frequencia[0] = store_type_frequency[0]
table_frequency.Frequencia[1] = store_type_frequency[3]
table_frequency.Frequencia[2] = store_type_frequency[2]
table_frequency.Frequencia[3] = store_type_frequency[1]
table_frequency.Frequencia[4] = store_type_frequency.sum()

table_frequency.Frequencia_Relativa[0] = round(store_type_frequency[0] / table_frequency.Frequencia[4] * 100,2)
table_frequency.Frequencia_Relativa[1] = round(store_type_frequency[3] / table_frequency.Frequencia[4] * 100,2)
table_frequency.Frequencia_Relativa[2] = round(store_type_frequency[2] / table_frequency.Frequencia[4] * 100,2)
table_frequency.Frequencia_Relativa[3] = round(store_type_frequency[1] / table_frequency.Frequencia[4] * 100,2)
table_frequency.Frequencia_Relativa[4] = table_frequency.Frequencia_Relativa.sum()

table_frequency

,Frequencia,Frequencia_Relativa
A,551627,54.23
B,15830,1.56
C,136840,13.45
D,312912,30.76
TOTAL,1017209,100


In [38]:
# 'assortment'

In [39]:
assortment_frequency = cat_attributes.assortment.value_counts()

In [40]:
assortment_frequency

a    537445
c    471470
b      8294
Name: assortment, dtype: int64

In [41]:
table_frequency = pd.DataFrame(columns=['Frequencia', 'Frequencia_Relativa'], index=['Basic', 'Extra', 'Extended','TOTAL'])

table_frequency.Frequencia[0] = assortment_frequency[0]
table_frequency.Frequencia[1] = assortment_frequency[2]
table_frequency.Frequencia[2] = assortment_frequency[1]
table_frequency.Frequencia[3] = assortment_frequency.sum()

table_frequency.Frequencia_Relativa[0] = round(assortment_frequency[0] / table_frequency.Frequencia[3] * 100,2)
table_frequency.Frequencia_Relativa[1] = round(assortment_frequency[2] / table_frequency.Frequencia[3] * 100,2)
table_frequency.Frequencia_Relativa[2] = round(assortment_frequency[1] / table_frequency.Frequencia[3] * 100,2)
table_frequency.Frequencia_Relativa[3] = table_frequency.Frequencia_Relativa.sum()

table_frequency

,Frequencia,Frequencia_Relativa
Basic,537445,52.84
Extra,8294,0.82
Extended,471470,46.35
TOTAL,1017209,100.01


In [42]:
# 'promo_interval'

In [43]:
promo_interval_frequency = cat_attributes.promo_interval.value_counts()

In [44]:
promo_interval_frequency

0                   508031
Jan,Apr,Jul,Oct     293122
Feb,May,Aug,Nov     118596
Mar,Jun,Sept,Dec     97460
Name: promo_interval, dtype: int64

In [45]:
table_frequency = pd.DataFrame(columns=['Frequencia', 'Frequencia_Relativa'], index=['Nenhuma', 'Jan-Apr-Jul-Oct', 'Feb-May-Aug-Nov','Mar-Jun-Sept-Dec','TOTAL'])

table_frequency.Frequencia[0] = promo_interval_frequency[0]
table_frequency.Frequencia[1] = promo_interval_frequency[1]
table_frequency.Frequencia[2] = promo_interval_frequency[2]
table_frequency.Frequencia[3] = promo_interval_frequency[3]
table_frequency.Frequencia[4] = promo_interval_frequency.sum()

table_frequency.Frequencia_Relativa[0] = round(promo_interval_frequency[0] / table_frequency.Frequencia[4] * 100,2)
table_frequency.Frequencia_Relativa[1] = round(promo_interval_frequency[1] / table_frequency.Frequencia[4] * 100,2)
table_frequency.Frequencia_Relativa[2] = round(promo_interval_frequency[2] / table_frequency.Frequencia[4] * 100,2)
table_frequency.Frequencia_Relativa[3] = round(promo_interval_frequency[3] / table_frequency.Frequencia[4] * 100,2)
table_frequency.Frequencia_Relativa[4] = table_frequency.Frequencia_Relativa.sum()

table_frequency

KeyError: 1